In [17]:
import re
import os
import csv
import nltk
print(nltk.__version__)
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from itertools import groupby
import pandas as pd

model = '/Library/Java/Extensions/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = '/Library/Java/Extensions/stanford-ner-2020-11-17/stanford-ner.jar'

st = StanfordNERTagger(model, jar,encoding='utf-8')


3.7


In [7]:
allData=[]

with open('articles.csv', newline='') as f:
    df = pd.read_csv(f)
    allData=df.values.tolist()

allName=[]
allOverView=[]
for i in allData:
    allName=allName+[i[0]]
    allOverView=allOverView+[i[1]]


In [8]:
orgList=[]
counter=0

while counter!= len(allOverView):
    
    if(isinstance(allOverView[counter], float) == False):
        text=allOverView[counter]

        tokenized_text = nltk.word_tokenize(text)
        classified_text = st.tag(tokenized_text)
        entities = []
        labels = []
        for tag, chunk in groupby(classified_text, lambda x:x[1]):
            if tag != "O":
                entities.append(' '.join(w for w, t in chunk))
                labels.append(tag)
        
        entities_unique = list(set(zip(entities, labels))) #unique entities  
        org=""
        for x in entities_unique:
            if(org==""):
                org=org+str(x)
            else:
                org=org+", "+str(x)
                

        orgList.insert(counter,org)
    else:
        orgList.insert(counter,"")
    counter=counter+1


In [9]:
print(orgList)

["('Lester', 'PERSON'), ('Grand County', 'LOCATION'), ('Colorado Sun', 'ORGANIZATION'), ('Larimer', 'LOCATION'), ('Colorado', 'LOCATION'), ('U.S. Forest Service', 'ORGANIZATION'), ('Continental', 'ORGANIZATION'), ('Cameron Peak', 'PERSON'), ('Utah State University', 'ORGANIZATION'), ('U.S.', 'LOCATION'), ('Colorado State Forest Service', 'ORGANIZATION'), ('Rockies', 'LOCATION'), ('Bridget Kochel', 'PERSON'), ('Colorado Parks and Wildlife', 'ORGANIZATION'), ('Mike Lester', 'PERSON'), ('Noel Livingston', 'PERSON'), ('Livingston', 'PERSON')", "('Colorado Sun', 'ORGANIZATION'), ('Larimer County', 'LOCATION'), ('Estes Valley', 'LOCATION'), ('Fern Lake', 'LOCATION'), ('Lewelling', 'PERSON'), ('Cameron Peak', 'PERSON'), ('Fort Collins', 'LOCATION'), ('Estes Park', 'LOCATION'), ('Green Mountain', 'LOCATION'), ('Beaver Meadows', 'PERSON'), ('Trail Ridge Road', 'LOCATION'), ('Colorado River', 'LOCATION'), ('Cameron', 'PERSON'), ('Mike Lewelling', 'PERSON'), ('National Weather Service', 'ORGANIZA

In [10]:
file = open('articlesExtracted.csv', 'w', newline ='')
allAttri=['Article Name','NER Name Extracted']
allRow=[allName, orgList]

with file:
    # identifying header  
    writer = csv.DictWriter(file, fieldnames = allAttri)
    writer.writeheader()
    counter=0
    # writing data row-wise into the csv file
    #goes for number of article times
    for i in range(len(allName)):
        inci={}
        for j in range(len(allAttri)):
            inci[allAttri[j]]=allRow[j][i]
        writer.writerow(inci)

Directory  outputTables  already exists


In [87]:
def stripKeyWords(NERout, ReqWordType):
    splitWords=['LOCATION','PERSON','ORGANIZATION']
    x = NERout.split()
    
    listFiltered=[]
    if ReqWordType == 'ALL':
        for i in range(len(x)):
            filtered=""
            ##if found the required word type
            if(x[i] in splitWords):
                j=i-1
                ##look for the previous key words(one of the splitting words)
                ##put the index in j
                while x[j] not in splitWords:
                    j=j-1
                ##add the words to filtered
                while j<i-1:
                    j=j+1
                    if filtered == '':
                        filtered=x[j]
                    else:
                        filtered=filtered+' '+x[j]
                ##add to a list of filtered words
                listFiltered=listFiltered+[filtered]

    else:
        for i in range(len(x)):
            filtered=""
            ##if found the required word type
            if(x[i] == ReqWordType):
                j=i-1
                ##look for the previous key words(one of the splitting words)
                ##put the index in j
                while x[j] not in splitWords and j !=0:
                    j=j-1
                ##add the words to filtered
                while j<i-1:
                    j=j+1
                    if filtered == '':
                        filtered=x[j]
                    else:
                        filtered=filtered+' '+x[j]
                ##add to a list of filtered words
                listFiltered=listFiltered+[filtered]
    return listFiltered

In [12]:
with open('articlesExtracted.csv', newline='') as f:
    df = pd.read_csv(f)

In [99]:
newTable=[]
for index,row in df.iterrows():
    dataFramObj=[]
    dataFramObj= dataFramObj+[row['Article Name']]
    if(isinstance(df.iloc[index]['NER Name Extracted'], float) == False):
        separateNames=df.iloc[index]['NER Name Extracted']
        separateNames=re.sub("[(]","",separateNames)
        separateNames=re.sub("[)]","",separateNames)
        separateNames=re.sub("[,]"," ",separateNames)
        separateNames=re.sub("[']","",separateNames)
        listFiltered=stripKeyWords(separateNames,'ALL')
        for i in listFiltered:
            if i != '':
                dataFramObj= dataFramObj+[i]
    newTable=newTable + [dataFramObj]


In [100]:
newTable

[['2020s_wildfire_season_is_historic_charring_more__Colorado_Sun_The_CO___October_26_2020.txt',
  'Lester',
  'Grand County',
  'Colorado Sun',
  'Larimer',
  'Colorado',
  'U.S. Forest Service',
  'Continental',
  'Cameron Peak',
  'Utah State University',
  'U.S.',
  'Colorado State Forest Service',
  'Rockies',
  'Bridget Kochel',
  'Colorado Parks and Wildlife',
  'Mike Lester',
  'Noel Livingston',
  'Livingston'],
 ['20_years_of_planning_saved_Rocky_Mountain_National__Colorado_Sun_The_CO___June_20_2021.txt',
  'Colorado Sun',
  'Larimer County',
  'Estes Valley',
  'Fern Lake',
  'Lewelling',
  'Cameron Peak',
  'Fort Collins',
  'Estes Park',
  'Green Mountain',
  'Beaver Meadows',
  'Trail Ridge Road',
  'Colorado River',
  'Cameron',
  'Mike Lewelling',
  'National Weather Service',
  'Michael Booth Early',
  'Glen Haven',
  'Cub Lake',
  'Mount Wuh',
  'Buthe',
  'Never Summer Range',
  'Forest Canyon Overlook on Trail Ridge Road',
  'Continental',
  'Colorado',
  'Bear Lake 

In [133]:
##extracting the column names
listColumnNames=['name/key words']
for i in newTable:
    for j in i:
        if j!=i[0] and j not in listColumnNames:
            listColumnNames=listColumnNames+[j]




In [131]:
print(listColumnNames)

['name/key words', 'Lester', 'Grand County', 'Colorado Sun', 'Larimer', 'Colorado', 'U.S. Forest Service', 'Continental', 'Cameron Peak', 'Utah State University', 'U.S.', 'Colorado State Forest Service', 'Rockies', 'Bridget Kochel', 'Colorado Parks and Wildlife', 'Mike Lester', 'Noel Livingston', 'Livingston', 'Larimer County', 'Estes Valley', 'Fern Lake', 'Lewelling', 'Fort Collins', 'Estes Park', 'Green Mountain', 'Beaver Meadows', 'Trail Ridge Road', 'Colorado River', 'Cameron', 'Mike Lewelling', 'National Weather Service', 'Michael Booth Early', 'Glen Haven', 'Cub Lake', 'Mount Wuh', 'Buthe', 'Never Summer Range', 'Forest Canyon Overlook on Trail Ridge Road', 'Bear Lake Road', 'Koren Nydick', 'Old Fall River Road', 'Rocky Mountain Park', 'Instagram', 'Beaver Mountain', 'Grand Lake', 'Doug', 'Spruce Canyon', 'Nydick', 'Darla Sidles', 'Youth Conservation Corps', 'Bear Lake', 'Upper Beaver Meadows Road', 'Moraine Park', 'Longmont', 'Parker', 'Onahu Lodge', 'Kawuneeche', 'Corral Creek 

In [132]:
##initializing a list of all rows
allRows=[]
##initialize allRows with numFire*numAttribute size
for i in newTable:
    row=[i[0]]
    for j in listColumnNames:
        row=row+[0]
    allRows=allRows+[row]

In [134]:
#add the ones
print(len(listColumnNames))
for i in range(len(newTable)):
    j=0
    while j<len(newTable[i])-1:
        j=j+1
        if(newTable[i][j]!=allRows[i][0] and newTable[i][j]!=''):
            allRows[i][listColumnNames.index(newTable[i][j])]=1
    
    
    

3680


In [135]:
file = open('ArticlesOutputTable.csv', 'w', newline ='')

with file:
    # identifying header  
    writer = csv.DictWriter(file, fieldnames = listColumnNames)
    writer.writeheader()

    # writing data row-wise into the csv file
    #goes for number of fire time
    for i in range(len(allRows)):
        inci={}
        for j in range(len(listColumnNames)):
            inci[listColumnNames[j]]=allRows[i][j]
        writer.writerow(inci)